In [ ]:
import pandas as pd

# Example DataFrame, replace this with your actual DataFrame
homer_df = pd.DataFrame(d_2)

# Process the texts with SpaCy
raw_texts = homer_df['unannotated_strings'].tolist()
annotated_texts = list(nlp.pipe(raw_texts, batch_size=100))

# Store the tokens in a new column, omitting spaces and punctuation
homer_df['tokens'] = [[token.text for token in doc if not token.is_space and not token.is_punct] for doc in annotated_texts]

homer_df['nlp_docs'] = list(annotated_texts)
# Display the DataFrame to check the results
homer_df

code to look at parent/children tokens:

for s in homer_df['nlp_docs']:  # Iterate over each SpaCy document
   for token in s:  # Iterate over each token in the document
       if not token.is_stop:  # Only process tokens that are not stop words
           # Get the parent token
           parent_token = token.head.text if token.head else None
           # Get the children tokens
           children_tokens = [c.text for c in token.children]
           
           # Print the token, its parent, and its children
           print(f"Token: {token.text}, Parent: {parent_token}, Children: {children_tokens}")

### Small Project

Dispersion analysis project
Comparing Pausanias and Herodotus

### Assembling a Corpus

The first step to this project will be assembling a small corpus for analysis. Given that we are interested in the linguistic differences between Pausanias and Herodotus, we need to find the URN for Herodotus's histories: urn:cts:greekLit:tlg0016.tlg001.perseus-grc2

Once we have the URN, we can fetch the file we need. 

Let's start with some exploration into each dataset. 

Like how we did in Week 02, we'll use MyCapytain and Pandas to open up each dataset and turn them into a format we can work with.

In [6]:
from MyCapytain.resources.texts.local.capitains.cts import CapitainsCtsText

with open("../tei/tlg0525.tlg001.perseus-grc2.xml") as f:
    pausanias_text = CapitainsCtsText(urn="urn:cts:greekLit:tlg0525.tlg001.perseus-grc2", resource=f)

with open("./tlg0016.tlg001.perseus-grc2.xml") as f:
    herodotus_text = CapitainsCtsText(urn="urn:cts:greekList:tlg0016.tlg001.perseus-grc2", resource=f)

with open("./tlg0012.tlg002.perseus-eng3.xml") as f:
    odyssey = CapitainsCtsText(urn="urn:cts:greekList:tlg0012.tlg002.perseus-eng3.xml", resource=f)

In [3]:
from lxml import etree
from MyCapytain.common.constants import Mimetypes


#pausanias
urns = []
raw_xmls = []
unannotated_strings = []

for ref in pausanias_text.getReffs(level=len(pausanias_text.citation)):
    urn = f"{pausanias_text.urn}:{ref}"
    node = pausanias_text.getTextualNode(ref)
    raw_xml = node.export(Mimetypes.XML.TEI)
    tree = node.export(Mimetypes.PYTHON.ETREE)
    s = etree.tostring(tree, encoding="unicode", method="text")

    urns.append(urn)
    raw_xmls.append(raw_xml)
    unannotated_strings.append(s)

import pandas as pd

d = {
    "urn": pd.Series(urns, dtype="string"),
    "raw_xml": raw_xmls,
    "unannotated_strings": pd.Series(unannotated_strings, dtype="string")
}
pausanias_df = pd.DataFrame(d)

#herodotus
urns = []
raw_xmls = []
unannotated_strings = []

for ref in herodotus_text.getReffs(level=len(herodotus_text.citation)):
    urn = f"{herodotus_text.urn}:{ref}"
    node = herodotus_text.getTextualNode(ref)
    raw_xml = node.export(Mimetypes.XML.TEI)
    tree = node.export(Mimetypes.PYTHON.ETREE)
    s = etree.tostring(tree, encoding="unicode", method="text")

    urns.append(urn)
    raw_xmls.append(raw_xml)
    unannotated_strings.append(s)

d = {
    "urn": pd.Series(urns, dtype="string"),
    "raw_xml": raw_xmls,
    "unannotated_strings": pd.Series(unannotated_strings, dtype="string")
}
herodotus_df = pd.DataFrame(d)



In [8]:
from lxml import etree
from MyCapytain.common.constants import Mimetypes
import pandas as pd
#homer
urns = []
raw_xmls = []
unannotated_strings = []

for ref in odyssey.getReffs(level=len(odyssey.citation)):
    urn = f"{odyssey.urn}:{ref}"
    node = odyssey.getTextualNode(ref)
    raw_xml = node.export(Mimetypes.XML.TEI)
    tree = node.export(Mimetypes.PYTHON.ETREE)
    s = etree.tostring(tree, encoding="unicode", method="text")

    urns.append(urn)
    raw_xmls.append(raw_xml)
    unannotated_strings.append(s)

d = {
    "urn": pd.Series(urns, dtype="string"),
    "raw_xml": raw_xmls,
    "unannotated_strings": pd.Series(unannotated_strings, dtype="string")
}
homer_df = pd.DataFrame(d)

In [9]:
import spacy 
spacy.cli.validate()

!python -m spacy download en


ading compatibility table...

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.7.6) =================
ℹ spaCy installation:
/workspaces/intro-text-analysis/.venv/lib/python3.11/site-packages/spacy

NAME             SPACY            VERSION                            
grc_proiel_trf   >=3.7.5,<3.8.0   3.7.5   ✔
grc_proiel_sm    >=3.7.5,<3.8.0   3.7.5   ✔

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.9 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [11]:
# Process the texts with SpaCy
import spacy
nlp = spacy.load("en_core_web_sm", disable=["ner"])

raw_texts = homer_df['unannotated_strings'].tolist()
annotated_texts = list(nlp.pipe(raw_texts, batch_size=100))

# Store the tokens in a new column, omitting spaces and punctuation
homer_df['tokens'] = [[token.text for token in doc if not token.is_space and not token.is_punct] for doc in annotated_texts]

homer_df['nlp_docs'] = list(annotated_texts)
# Display the DataFrame to check the results
homer_df



,urn,raw_xml,unannotated_strings,tokens,nlp_docs
0,urn:cts:greekList:tlg0012.tlg002.perseus-eng3....,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...","Tell me, O Muse, of the man of many devices, w...","[Tell, me, O, Muse, of, the, man, of, many, de...","(Tell, me, ,, O, Muse, ,, of, the, man, of, ma..."
1,urn:cts:greekList:tlg0012.tlg002.perseus-eng3....,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...","Then the goddess, flashing-eyed1 Athena, ...","[Then, the, goddess, flashing, eyed1, Athena, ...","(\n , Then, the, goddess, ,, flashing, -, e..."
2,urn:cts:greekList:tlg0012.tlg002.perseus-eng3....,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...","Then the goddess, flashing-eyed Athena, answer...","[Then, the, goddess, flashing, eyed, Athena, a...","(Then, the, goddess, ,, flashing, -, eyed, Ath..."
3,urn:cts:greekList:tlg0012.tlg002.perseus-eng3....,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...","So saying, he led the way, and Pallas Athena f...","[So, saying, he, led, the, way, and, Pallas, A...","(So, saying, ,, he, led, the, way, ,, and, Pal..."
4,urn:cts:greekList:tlg0012.tlg002.perseus-eng3....,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...","Then the goddess, flashing-eyed Athena, a...","[Then, the, goddess, flashing, eyed, Athena, a...","(\n , Then, the, goddess, ,, flashing, -, e..."
...,...,...,...,...,...
283,urn:cts:greekList:tlg0012.tlg002.perseus-eng3....,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...","Then Laertes answered him again, and said...","[Then, Laertes, answered, him, again, and, sai...","(\n , Then, Laertes, answered, him, again, ..."
284,urn:cts:greekList:tlg0012.tlg002.perseus-eng3....,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...",Meanwhile the Sicilian handmaid bathed great-h...,"[Meanwhile, the, Sicilian, handmaid, bathed, g...","(Meanwhile, the, Sicilian, handmaid, bathed, g..."
285,urn:cts:greekList:tlg0012.tlg002.perseus-eng3....,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...",So they were busied with their meal in th...,"[So, they, were, busied, with, their, meal, in...","(\n , So, they, were, busied, with, their, ..."
286,urn:cts:greekList:tlg0012.tlg002.perseus-eng3....,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...","So he spoke, and thereat pale fear seized them...","[So, he, spoke, and, thereat, pale, fear, seiz...","(So, he, spoke, ,, and, thereat, pale, fear, s..."


In [16]:
#code to look at parent/children tokens:

for s in homer_df['nlp_docs']:  # Iterate over each SpaCy document
   for token in s:  # Iterate over each token in the document
       if not token.is_stop and token.is_alpha: #not token.text == ",":  # Only process tokens that are not stop words
           # Get the parent token
           parent_token = token.head.text if token.head else None
           # Get the children tokens
           children_tokens = [c.text for c in token.children]
           
           # Print the token, its parent, and its children
           print(f"Token: {token.text}, Parent: {parent_token}, Children: {children_tokens}")

Token: Tell, Parent: Tell, Children: ['me', ',', 'Muse', '.']
Token: O, Parent: Muse, Children: []
Token: Muse, Parent: Tell, Children: ['O', ',', 'of']
Token: man, Parent: of, Children: ['the', 'of']
Token: devices, Parent: of, Children: ['many', ',', 'wandered']
Token: wandered, Parent: devices, Children: ['who', 'ways', 'sacked']
Token: ways, Parent: wandered, Children: ['full', 'many']
Token: sacked, Parent: wandered, Children: ['after', 'he', 'had', 'citadel']
Token: sacred, Parent: citadel, Children: []
Token: citadel, Parent: sacked, Children: ['the', 'sacred', 'of']
Token: Troy, Parent: of, Children: []
Token: men, Parent: were, Children: ['the', 'saw']
Token: cities, Parent: saw, Children: ['whose']
Token: saw, Parent: men, Children: ['cities', 'he', 'and', 'learned']
Token: mind, Parent: learned, Children: ['whose']
Token: learned, Parent: saw, Children: ['mind', 'he', ',', 'aye']
Token: aye, Parent: learned, Children: [',', 'and', 'woes']
Token: woes, Parent: aye, Children: 

### Counting tokens, types, etc

With dataframes for each work created, let's get an idea of how many tokens are in each corpus.

In [ ]:
pausanias_df['whitespaced_tokens'] = pausanias_df['unannotated_strings'].str.split()
pausanias_df['whitespaced_tokens'].explode().count()

herodotus_df['whitespaced_tokens'] = herodotus_df['unannotated_strings'].str.split()
herodotus_df['whitespaced_tokens'].explode().count()


Okay, so we have an idea of the number of tokens in each corpus. Pausanias has 
In week 3, we worked on counting the number of occurences of the definite article in Pausanias using an nlp pipeline, then determining the relative frequency of the definite article.. Let's repeat these steps for both Pausanias and Herodotus, just to explore the data a little further. 

In [ ]:
import spacy

nlp = spacy.load("grc_proiel_sm", disable=["ner"])

#pausanias
raw_texts_p = [t for t in pausanias_df['unannotated_strings']]
annotated_texts_p = nlp.pipe(raw_texts_p, batch_size=100)


#herodotus
raw_texts_h= [t for t in herodotus_df['unannotated_strings']]
annotated_texts_h = nlp.pipe(raw_texts_h, batch_size=100)



In [ ]:
#pausanias
pausanias_df['nlp_docs'] = list(annotated_texts_p)
definite_article_p = [t for t in pausanias_df['nlp_docs'].explode() if t.lemma_ == "ὁ"]
n_def_article_p = len(definite_article_p)
n_tokens_p = len([t for t in pausanias_df['nlp_docs'].explode()])

#herodotus
herodotus_df['nlp_docs'] = list(annotated_texts_h)
definite_article_h = [t for t in herodotus_df['nlp_docs'].explode() if t.lemma_ == "ὁ"]
n_def_article_h = len(definite_article_h)
n_tokens_h = len([t for t in herodotus_df['nlp_docs'].explode()])

basis = 10_000

rf_definite_article_in_pausanias = (n_def_article_p / n_tokens_p) * basis
rf_definite_article_in_herodotus = (n_def_article_h / n_tokens_h) * basis

print(rf_definite_article_in_pausanias)
print(rf_definite_article_in_herodotus)

Hm... why the difference? Surely we can't expect the relative frequency of the definite article to be the exact same in two different works. But Pausanias does use the deinite article more, so I'm curious whether this is statistically significant. 